In [3]:
import zipfile
import os


with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")


os.listdir("/content/")

['.config', 'GPUPy.zip', 'GPUPy', 'sample_data']

In [4]:
import sys
sys.path.append("/content")

In [14]:
import GPUPy
from GPUPy.src.numerical_methods.root_finding import bisection
import time

# Define a custom benchmark function for this specific case
def custom_benchmark(method, func, a, b, repeats=5, **kwargs):
    """Custom benchmark that verifies sign change before each call."""
    # Verify sign change
    fa = func(a)
    fb = func(b)
    if fa * fb >= 0:
        raise ValueError("Function values at interval endpoints must have opposite signs.")

    # Now time the method calls
    durations = []
    for _ in range(repeats):
        start = time.perf_counter()
        method(func, a, b, **kwargs)
        durations.append(time.perf_counter() - start)

    avg_time = sum(durations) / repeats
    return avg_time

# Simple polynomial with guaranteed sign change
def simple_poly(x):
    return x**3 - 2*x - 5  # f(-2) < 0, f(3) > 0

# Try with custom benchmark
cpu_time = custom_benchmark(
    bisection,
    simple_poly,
    a=-2,
    b=3,
    tolerance=1e-6,
    max_iterations=100,
    use_gpu=False
)

gpu_time = custom_benchmark(
    bisection,
    simple_poly,
    a=-2,
    b=3,
    tolerance=1e-6,
    max_iterations=100,
    use_gpu=True
)

print(f"Simple Poly Bisection CPU Time: {cpu_time:.6f} seconds")
print(f"Simple Poly Bisection GPU Time: {gpu_time:.6f} seconds")

Simple Poly Bisection CPU Time: 0.000129 seconds
Simple Poly Bisection GPU Time: 0.009026 seconds


In [18]:
from GPUPy.src.numerical_methods.root_finding import newton_raphson
import numpy as np
# Sinüs fonksiyonları
def sin_func(x):
    return np.sin(x)

def d_sin_func(x):
    return np.cos(x)

# CPU time
cpu_time_sin = GPUPy.benchmark(newton_raphson, sin_func, d_sin_func, x0=3, tol=1e-6, max_iter=100, use_gpu=False)

# GPU time
gpu_time_sin = GPUPy.benchmark(newton_raphson, sin_func, d_sin_func, x0=3, tol=1e-6, max_iter=100, use_gpu=True)

print(f"Sin Function Newton-Raphson CPU Time: {cpu_time_sin:.6f} seconds")
print(f"Sin Function Newton-Raphson GPU Time: {gpu_time_sin:.6f} seconds")

Sin Function Newton-Raphson CPU Time: 0.000022 seconds
Sin Function Newton-Raphson GPU Time: 0.001254 seconds


In [19]:
# Exponential-logarithmic fonksiyon
def exp_log_func(x):
    return np.exp(x) - np.log(x + 2)

# CPU time
cpu_time_exp_log = GPUPy.benchmark(bisection, exp_log_func, a=-1, b=2, tolerance=1e-6, max_iterations=100, use_gpu=False)

# GPU time
gpu_time_exp_log = GPUPy.benchmark(bisection, exp_log_func, a=-1, b=2, tolerance=1e-6, max_iterations=100, use_gpu=True)

print(f"Exp-Log Function Bisection CPU Time: {cpu_time_exp_log:.6f} seconds")
print(f"Exp-Log Function Bisection GPU Time: {gpu_time_exp_log:.6f} seconds")

ValueError: Function values at interval endpoints must have opposite signs.

# Yeni Bölüm